In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -U transformers evaluate

In [3]:
import torch

torch.backends.cudnn.deterministic = True

RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

cuda


In [8]:
import requests
import shutil
import gzip
import pandas as pd

url = ("https://github.com/rasbt/machine-learning-book/raw/main/ch08/movie_data.csv.gz")
filename = url.split("/")[-1]
with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)
with gzip.open('movie_data.csv.gz', 'rb') as f_in:
    with open('movie_data.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

df = pd.read_csv('movie_data.csv')
print(df.head(3))

                                              review  sentiment
0  In 1974, the teenager Martha Moxley (Maggie Gr...          1
1  OK... so... I really like Kris Kristofferson a...          0
2  ***SPOILER*** Do not read this, if you think a...          0


In [9]:
train_texts = df.iloc[:35000]['review'].values
train_labels = df.iloc[:35000]['sentiment'].values
valid_texts = df.iloc[35000:40000]['review'].values
valid_labels = df.iloc[35000:40000]['sentiment'].values
test_texts = df.iloc[40000:]['review'].values
test_labels = df.iloc[40000:]['sentiment'].values
print(train_texts.shape)
print(train_labels.shape)

(35000,)
(35000,)


In [6]:
import tokenizers
print(tokenizers.__version__)
import transformers
print(transformers.__version__)


from transformers import AutoTokenizer
from transformers import DistilBertForSequenceClassification


MODEL = f"distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

0.22.1
4.57.1


2025-10-24 18:13:44.522776: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761329624.544907     225 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761329624.551952     225 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
valid_encodings = tokenizer(list(valid_texts), truncation=True, padding=True)
# test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

In [23]:
print({key: torch.tensor(val[0]) for key, val in train_encodings.items()})

{'input_ids': tensor([  101,  1999,  3326,  1010,  1996, 10563,  9246,  9587, 20959,  1006,
         8538,  4519,  1007,  5829,  2000,  1996,  2152,  1011,  2465,  2181,
         1997,  9852,  4033,  1010, 13861,  1010,  6117,  1012,  2006,  1996,
        25166,  2305,  1010,  6574,  1997, 14414,  1010,  2016,  2001,  7129,
         1999,  1996, 16125,  1997,  2014,  2160,  1998,  2014,  4028,  2815,
         4895, 19454,  7178,  1012,  3174,  1011,  2048,  2086,  2101,  1010,
         1996,  3213,  2928, 11865,  8093,  2386,  1006,  5696, 11463, 10698,
         1007,  1010,  2040,  2003,  1037,  2280,  2474,  6317,  2008,  2038,
         5357,  1999, 29591,  2005,  2566,  9103,  2854,  1999,  1051,  1012,
         1046,  1012,  9304,  3979,  1998,  2333,  2000,  9795,  1010,  7288,
         2000,  8556,  1996,  2553,  2007,  2010,  4256,  4459,  3134,  1006,
         4080,  6395,  1007,  2007,  1996,  3800,  1997,  3015,  1037,  2338,
         1012,  1996, 10575,  5490, 10179, 10867, 

In [27]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx], device=DEVICE) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], device=DEVICE)
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
valid_dataset = IMDbDataset(valid_encodings, valid_labels)
# test_dataset = IMDbDataset(test_encodings, test_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True) 
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False) 
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

In [25]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(DEVICE)
model.train()
optim = torch.optim.Adam(model.parameters(), lr=5e-5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
import evaluate
import numpy as np
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [28]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='./results', num_train_epochs=3, 
    per_device_train_batch_size=16, per_device_eval_batch_size=16, 
    logging_dir='./logs', logging_steps=10
)
trainer = Trainer(
    model=model, args=training_args, 
    train_dataset=train_dataset, eval_dataset=valid_dataset, 
    optimizers=(optim, None), compute_metrics=compute_metrics,
)


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [ ]:
trainer.train()